<a href="https://colab.research.google.com/github/mengcius/pytorch-learn/blob/master/11_%E8%BF%87%E6%8B%9F%E5%90%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 11_过拟合

### 划分train-val-test集来训练
 可将原来的train集划分为train集和val集，test集不变。
 
 train集用来训练，val集用来挑选模型参数，test仅仅用来评价。

In [1]:
import  torch
import  torch.nn as nn
import  torch.nn.functional as F
import  torch.optim as optim
from    torchvision import datasets, transforms

batch_size=200
learning_rate=0.01
epochs=10

# 加载数据，划分train、val、test集
train_db = datasets.MNIST('mnist_data', train=True, download=True,
            transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))]))     
# train_loader = torch.utils.data.DataLoader(train_db, batch_size=batch_size, shuffle=True)

test_db = datasets.MNIST('mnist_data', train=False, download=True, #train=False，不训练
            transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))]))
test_loader = torch.utils.data.DataLoader(test_db, batch_size=batch_size, shuffle=True)

print('train:', len(train_db), 'test:', len(test_db))
train_db, val_db = torch.utils.data.random_split(train_db, [50000, 10000]) #将原train集随机划分为train和val集
print('train_db:', len(train_db), 'val_db:', len(val_db))

train_loader = torch.utils.data.DataLoader(train_db, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_db, batch_size=batch_size, shuffle=True)


# 模型构建
class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(784, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 10),
            nn.LeakyReLU(inplace=True),
        )

    def forward(self, x):
        x = self.model(x)

        return x


# train集训练
device = torch.device('cuda:0')
net = MLP().to(device)
optimizer = optim.SGD(net.parameters(), lr=learning_rate)
criteon = nn.CrossEntropyLoss().to(device)

for epoch in range(epochs):

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28)
        data, target = data.to(device), target.cuda()

        logits = net(data)
        loss = criteon(logits, target)

        optimizer.zero_grad()
        loss.backward()
        # print(w1.grad.norm(), w2.grad.norm())
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


    # val集验证，以挑选出最优的模型
    test_loss = 0
    correct = 0
    for data, target in val_loader:
        data = data.view(-1, 28 * 28)
        data, target = data.to(device), target.cuda()
        logits = net(data)
        test_loss += criteon(logits, target).item()

        pred = logits.data.max(1)[1]
        correct += pred.eq(target.data).sum()

    test_loss /= len(val_loader.dataset)
    print('\nVAL set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))


# test集测试评价
#...#这里可加载从val中找到的最佳的模型，这里默认用最后一个
test_loss = 0
correct = 0
for data, target in test_loader:
    data = data.view(-1, 28 * 28)
    data, target = data.to(device), target.cuda()
    logits = net(data)
    test_loss += criteon(logits, target).item()

    pred = logits.data.max(1)[1]
    correct += pred.eq(target.data).sum()

test_loss /= len(test_loader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

0it [00:00, ?it/s]

9920512it [00:01, 8760472.64it/s]                            


Extracting mnist_data/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 132007.50it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 2211362.04it/s]                           
0it [00:00, ?it/s]

Extracting mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 49619.67it/s]            


Extracting mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!
train: 60000 test: 10000
train_db: 50000 val_db: 10000
Train Epoch: 0 [0/50000 (0%)]	Loss: 2.289106
Train Epoch: 0 [20000/50000 (40%)]	Loss: 2.100112
Train Epoch: 0 [40000/50000 (80%)]	Loss: 1.335677

VAL set: Average loss: 0.0057, Accuracy: 7404/10000 (74%)

Train Epoch: 1 [0/50000 (0%)]	Loss: 1.145260
Train Epoch: 1 [20000/50000 (40%)]	Loss: 0.897504
Train Epoch: 1 [40000/50000 (80%)]	Loss: 0.676969

VAL set: Average loss: 0.0029, Accuracy: 8708/10000 (87%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 0.632673
Train Epoch: 2 [20000/50000 (40%)]	Loss: 0.424209
Train Epoch: 2 [40000/50000 (80%)]	Loss: 0.379384

VAL set: Average loss: 0.0019, Accuracy: 8961/10000 (89%)

Train Epoch: 3 [0/50000 (0%)]	Loss: 0.434680
Train Epoch: 3 [20000/50000 (40%)]	Loss: 0.304217
Train Epoch: 3 [40000/50000 (80%)]	Loss: 0.264263

VAL set: Average loss: 0.0016, Accuracy: 9065/10000 (90%)

Train Epoch: 4 [0/50000 (0%)]	Loss: 0.373092
Tr